In [1]:
%cd ../../../

/Users/nseverin/MyData/Projects/Science/LLM/sasrec-bert4rec-recsys23


/Users/nseverin/MyData/Projects/Science/LLM/sasrec-bert4rec-recsys23/venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from collections import defaultdict
import re
import numpy as np
import pandas as pd

def open_text(filename): 
    with open(filename) as f:
        return f.read()


def parse_file_llm(log_data):
    # Regex patterns
    config_pattern = re.compile(r"experiments(.+?\.yaml)")
    epoch_pattern = re.compile(r"Epoch (\d+)/\d+, Loss: ([\d.]+)")
    metrics_pattern = re.compile(r"(Validation|Test) Metrics: ({.+?})")
    
    # Parse the log
    results = defaultdict(lambda: {"validation_metrics": [], "test_metrics": []})
    current_config = None
        
    for line in log_data.splitlines():
        config_match = config_pattern.search(line)
        if config_match:
            current_config = config_match.group(1)
        
        epoch_match = epoch_pattern.search(line)
        if epoch_match:
            epoch_num = int(epoch_match.group(1))
            loss = float(epoch_match.group(2))
        
        metrics_match = metrics_pattern.search(line)
        if metrics_match and current_config:
            metric_type = metrics_match.group(1).lower()  # "validation" or "test"
            metrics = eval(metrics_match.group(2))  # Convert string to dict
            results[current_config][f"{metric_type}_metrics"].append({"epoch": epoch_num, "loss": loss, **metrics})
    return results


def find_max(results, config, metric='NDCG@10'):
    def get_dct_epoch(lst, epoch):
        for x in lst:
            if x['epoch'] == epoch:
                return x
        return lst[-1]
        # raise Exception()
    
    def postprocess(lst):
        res = []
        for dct in lst:
            res.append({
                'NDCG@10': round(dct['NDCG@10'], 5),
                'Recall@10': round(dct['Recall@10'], 5),
                'epoch': dct['epoch']
            })
        return res
    
    best_val_dct = None
    for dct in results[config]['validation_metrics']:
        if best_val_dct is None:
            best_val_dct = dct
        elif dct[metric] > best_val_dct[metric]:
            best_val_dct = dct
    
    best_val_epoch = best_val_dct['epoch']
    
    return {
        'test': postprocess([get_dct_epoch(results[config]['test_metrics'], best_val_epoch)]),
        'validation': postprocess([best_val_dct]),
    }


PARAMS = ["config_file", "weighting_scheme", "alpha", "fine_tune_epoch", 'reconstruct_loss', "reconstruction_layer", 'weight_scale', 'use_down_scale','use_upscale','multi_profile','multi_profile_aggr_scheme','scale_guide_loss','user_profile_embeddings_files']


def create_dataframe(exp_data_with_max_val):
    # List to store extracted experiment data
    data = []
    
    # Parse experiment results
    for config_path, metrics in exp_data_with_max_val.items():
        # Extract config details from the filename
        config_file = config_path.split("/")[-1]  # Get only the filename
        config_name = config_file.replace(".yaml", "").lstrip('sasrec-').split("-")
        
        
        # {weighting_scheme}-{alpha}-{fine_tune_epoch}-{reconstruct_loss}-{reconstruction_layer}-{weight_scale}-{use_down_scale}-{use_upscale}-{multi_profile}-{multi_profile_aggr_scheme}-{scale_guide_loss}-{user_profile_embeddings_files}-{seed}.yaml
        weighting_scheme = (config_name[0])
        alpha = float(config_name[1])
        fine_tune_epoch = int(config_name[2])
        reconstruct_loss = (config_name[3])
        reconstruction_layer = int(config_name[4])
        weight_scale = float(config_name[5])
        use_down_scale = (config_name[6])
        use_upscale = (config_name[7])
        multi_profile = (config_name[8])
        multi_profile_aggr_scheme = (config_name[9])
        scale_guide_loss = (config_name[10])
        user_profile_embeddings_files = (config_name[11])
        seed = int(config_name[12])
        
        # Extract validation and test metrics
        val_metrics = metrics.get("validation", [{}])[0]
        test_metrics = metrics.get("test", [{}])[0]
    
        # Append extracted values to data list
        data.append([
            config_file, weighting_scheme, alpha, fine_tune_epoch, reconstruct_loss, reconstruction_layer, weight_scale, use_down_scale,use_upscale,multi_profile,multi_profile_aggr_scheme,scale_guide_loss,user_profile_embeddings_files,seed,
            val_metrics.get("NDCG@10", None), val_metrics.get("Recall@10", None), val_metrics.get("epoch", None),
            test_metrics.get("NDCG@10", None), test_metrics.get("Recall@10", None), test_metrics.get("epoch", None)
        ])
    
    # Create DataFrame
    columns = PARAMS + ['seed',
        "val_NDCG@10", "val_Recall@10", "val_epoch", "test_NDCG@10", "test_Recall@10", "test_epoch"
        ]
    
    df = pd.DataFrame(data, columns=columns)
    return df


def apply_seed_info(df):
    df['params'] = df.apply(lambda row: tuple([row[param] for param in PARAMS[1:]]), axis=1)
    params2seeds = {}
    for params, seed in zip(df['params'], df['seed']):
        if params not in params2seeds:
            params2seeds[params] = []
        params2seeds[params].append(seed)
    df['all_seeds'] = df.apply(lambda row: (params2seeds[row['params']]), axis=1)
    return df

In [3]:
len('{weighting_scheme}-{alpha}-{fine_tune_epoch}-{reconstruct_loss}-{reconstruction_layer}-{weight_scale}-{use_down_scale}-{use_upscale}-{multi_profile}-{multi_profile_aggr_scheme}-{scale_guide_loss}-{user_profile_embeddings_files}-{seed}.yaml'.split('-'))

13

In [17]:
MODEL_NAME = 'sasrec'
DATASET = 'kion_en'
EXPERIMENT_NAME = 'KION_INITIAL'
SPLIT_NAME = 'general'
MODE = 'LLM'

local_directory = f"experiments-2_0/results/{MODEL_NAME}/{DATASET}/{EXPERIMENT_NAME}" 

seed_folders = ['single_seed']

In [18]:
import os


results = {}
for seed_folder in seed_folders:
    cur_folder = os.path.join(local_directory, seed_folder)
    for file in os.listdir(cur_folder):
        log_data = open_text(f'{cur_folder}/{file}')
        cur_results = parse_file_llm(log_data)
        # if cur_results['simple']['validation_metrics'] == []:
        #     continue
        results.update(cur_results)
    
    
exp_data_with_max_val = {}
for config in results:
    exp_data_with_max_val[config] = find_max(results, config, metric='NDCG@10')
exp_data_with_max_val.keys()

dict_keys(['-2_0/configs/sasrec/kion_en/KION_INITIAL/single_seed/attention-0.65-6-RMSE-1-0.1-True-False-False-mean-True-gemma_long_large_single-42.yaml', '-2_0/configs/sasrec/kion_en/KION_INITIAL/single_seed/attention-0.65-6-RMSE-1-0.1-True-False-False-mean-True-gemma_long_large_umap_single-42.yaml', '-2_0/configs/sasrec/kion_en/KION_INITIAL/single_seed/attention-0.65-6-RMSE-1-0.1-True-False-False-mean-True-gemma_short_large_single-42.yaml', '-2_0/configs/sasrec/kion_en/KION_INITIAL/single_seed/attention-0.65-6-RMSE-1-0.1-True-False-False-mean-True-gemma_short_large_umap_single-42.yaml', '-2_0/configs/sasrec/kion_en/KION_INITIAL/single_seed/attention-0.65-6-RMSE-2-0.1-False-True-False-mean-False-gemma_long_large_single-42.yaml', '-2_0/configs/sasrec/kion_en/KION_INITIAL/single_seed/attention-0.65-6-RMSE-2-0.1-False-True-False-mean-False-gemma_long_large_umap_single-42.yaml', '-2_0/configs/sasrec/kion_en/KION_INITIAL/single_seed/attention-0.65-6-RMSE-2-0.1-False-True-False-mean-False-ge

In [19]:
df = create_dataframe(exp_data_with_max_val)
df['weighting_scheme'] = df['weighting_scheme'].apply(lambda x: x if x != 'ttention' else 'attention')
df['weighting_scheme'] = df['weighting_scheme'].apply(lambda x: x if x != 'xponential' else 'exponential')
df

,config_file,weighting_scheme,alpha,fine_tune_epoch,reconstruct_loss,reconstruction_layer,weight_scale,use_down_scale,use_upscale,multi_profile,multi_profile_aggr_scheme,scale_guide_loss,user_profile_embeddings_files,seed,val_NDCG@10,val_Recall@10,val_epoch,test_NDCG@10,test_Recall@10,test_epoch
0,attention-0.65-6-RMSE-1-0.1-True-False-False-m...,attention,0.65,6,RMSE,1,0.1,True,False,False,mean,True,gemma_long_large_single,42,0.09815,0.18329,13,0.05931,0.11389,13
1,attention-0.65-6-RMSE-1-0.1-True-False-False-m...,attention,0.65,6,RMSE,1,0.1,True,False,False,mean,True,gemma_long_large_umap_single,42,0.09944,0.18416,11,0.05882,0.11406,11
2,attention-0.65-6-RMSE-1-0.1-True-False-False-m...,attention,0.65,6,RMSE,1,0.1,True,False,False,mean,True,gemma_short_large_single,42,0.09811,0.18351,13,0.05939,0.11452,13
3,attention-0.65-6-RMSE-1-0.1-True-False-False-m...,attention,0.65,6,RMSE,1,0.1,True,False,False,mean,True,gemma_short_large_umap_single,42,0.09881,0.18291,12,0.05725,0.11215,12
4,attention-0.65-6-RMSE-2-0.1-False-True-False-m...,attention,0.65,6,RMSE,2,0.1,False,True,False,mean,False,gemma_long_large_single,42,0.09899,0.18449,8,0.05799,0.11299,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,exponential-0.5-4-MSE-2-0.1-False-True-False-m...,exponential,0.50,4,MSE,2,0.1,False,True,False,mean,False,gemma_short_large_umap_single,42,0.09920,0.18161,8,0.05986,0.11466,8
1724,exponential-0.5-4-MSE-2-0.1-False-True-False-m...,exponential,0.50,4,MSE,2,0.1,False,True,False,mean,True,gemma_long_large_single,42,0.09952,0.18329,14,0.05756,0.11238,14
1725,exponential-0.5-4-MSE-2-0.1-False-True-False-m...,exponential,0.50,4,MSE,2,0.1,False,True,False,mean,True,gemma_long_large_umap_single,42,0.09871,0.18226,10,0.05935,0.11525,10
1726,exponential-0.5-4-MSE-2-0.1-False-True-False-m...,exponential,0.50,4,MSE,2,0.1,False,True,False,mean,True,gemma_short_large_single,42,0.09929,0.18481,19,0.05677,0.11155,19


In [20]:
df['weighting_scheme'].value_counts()

weighting_scheme
attention      576
mean           576
exponential    576
Name: count, dtype: int64

In [21]:
df = apply_seed_info(df)
df.sort_values(by='test_NDCG@10', ascending=False, inplace=True)
df

,config_file,weighting_scheme,alpha,fine_tune_epoch,reconstruct_loss,reconstruction_layer,weight_scale,use_down_scale,use_upscale,multi_profile,...,user_profile_embeddings_files,seed,val_NDCG@10,val_Recall@10,val_epoch,test_NDCG@10,test_Recall@10,test_epoch,params,all_seeds
663,mean-0.8-6-RMSE-1-0.1-False-True-False-mean-Fa...,mean,0.80,6,RMSE,1,0.1,False,True,False,...,gemma_short_large_umap_single,42,0.09987,0.18422,14,0.06111,0.11661,14,"(mean, 0.8, 6, RMSE, 1, 0.1, False, True, Fals...",[42]
837,exponential-0.8-8-MSE-2-0.1-False-True-False-m...,exponential,0.80,8,MSE,2,0.1,False,True,False,...,gemma_long_large_umap_single,42,0.09898,0.18188,14,0.06083,0.11617,14,"(exponential, 0.8, 8, MSE, 2, 0.1, False, True...",[42]
779,exponential-0.65-8-RMSE-2-0.1-False-True-False...,exponential,0.65,8,RMSE,2,0.1,False,True,False,...,gemma_short_large_umap_single,42,0.09998,0.18356,14,0.06072,0.11579,14,"(exponential, 0.65, 8, RMSE, 2, 0.1, False, Tr...",[42]
649,mean-0.8-6-MSE-2-0.1-False-True-False-mean-Tru...,mean,0.80,6,MSE,2,0.1,False,True,False,...,gemma_long_large_umap_single,42,0.09983,0.18389,14,0.06072,0.11615,14,"(mean, 0.8, 6, MSE, 2, 0.1, False, True, False...",[42]
467,exponential-0.5-8-RMSE-2-0.1-True-False-False-...,exponential,0.50,8,RMSE,2,0.1,True,False,False,...,gemma_short_large_umap_single,42,0.09985,0.18454,14,0.06072,0.11703,14,"(exponential, 0.5, 8, RMSE, 2, 0.1, True, Fals...",[42]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,exponential-0.8-8-RMSE-2-0.1-True-False-False-...,exponential,0.80,8,RMSE,2,0.1,True,False,False,...,gemma_long_large_single,42,0.09951,0.18465,19,0.05589,0.10964,19,"(exponential, 0.8, 8, RMSE, 2, 0.1, True, Fals...",[42]
792,exponential-0.8-4-MSE-1-0.1-False-True-False-m...,exponential,0.80,4,MSE,1,0.1,False,True,False,...,gemma_long_large_single,42,0.09851,0.18318,11,0.05586,0.10847,11,"(exponential, 0.8, 4, MSE, 1, 0.1, False, True...",[42]
25,attention-0.65-8-MSE-1-0.1-False-True-False-me...,attention,0.65,8,MSE,1,0.1,False,True,False,...,gemma_long_large_umap_single,42,0.09773,0.18112,18,0.05583,0.10835,18,"(attention, 0.65, 8, MSE, 1, 0.1, False, True,...",[42]
274,attention-0.8-6-MSE-2-0.1-True-False-False-mea...,attention,0.80,6,MSE,2,0.1,True,False,False,...,gemma_short_large_single,42,0.09796,0.18210,7,0.05579,0.10824,7,"(attention, 0.8, 6, MSE, 2, 0.1, True, False, ...",[42]


In [22]:
df.to_csv('kion_llm_all_runs.csv')

In [16]:
def get_stats_seeds(df_all):
    def aggregate_mean(series):
        first_elem = list(series)[0]
        if isinstance(first_elem, str) or isinstance(first_elem, bool):
            return first_elem
        return series.mean()
        # print(df)
        # all_cols = set(df.columns)
        # df_mean = df.mean(numeric_only=True)
        # rest_cols = all_cols - set(df_mean.columns)
        # for col in rest_cols:
        #     df_mean[col] = list(df[col])[0]
        # return df_mean
    
    def aggregate_std(series):
        # print(series)
        first_elem = list(series)[0]
        if isinstance(first_elem, str) or isinstance(first_elem, bool):
            return first_elem
        if len(series) == 1:
            return 0
        return series.std()
        # all_cols = set(df.columns)
        # df_mean = df.std(numeric_only=True)
        # rest_cols = all_cols - set(df_mean.columns)
        # for col in rest_cols:
        #     df_mean[col] = list(df[col])[0]
        # return df_mean
    
    
    metric_cols = ["val_NDCG@10", "val_Recall@10", "val_epoch", "test_NDCG@10", "test_Recall@10"]
        
    grouped_df = df_all.drop(['config_file', 'all_seeds', 'seed'], axis=1).groupby('params')
    df_mean = grouped_df.agg(aggregate_mean)
    df_std = grouped_df.agg(aggregate_std)
    for col in metric_cols:
        df_mean[col + '__std'] = df_std[col]
    return df_mean


def reorder_cols(df):
    order = ['val_epoch', 'val_epoch__std'] + PARAMS[1:] + ['val_NDCG@10', 'val_NDCG@10__std', 'val_Recall@10', 'val_Recall@10__std', 'test_NDCG@10', 'test_NDCG@10__std', 'test_Recall@10', 'test_Recall@10__std']
    return df[order]


df_all = df[df['all_seeds'].apply(len) == 3]
df_final = get_stats_seeds(df_all)
df_final = reorder_cols(df_final)
df_final.sort_values(by='test_NDCG@10', ascending=False, inplace=True)
df_final

,val_epoch,val_epoch__std,weighting_scheme,alpha,fine_tune_epoch,reconstruct_loss,reconstruction_layer,weight_scale,use_down_scale,use_upscale,...,scale_guide_loss,user_profile_embeddings_files,val_NDCG@10,val_NDCG@10__std,val_Recall@10,val_Recall@10__std,test_NDCG@10,test_NDCG@10__std,test_Recall@10,test_Recall@10__std
params,,,,,,,,,,,,,,,,,,,,,


In [44]:
df_final['weighting_scheme'].value_counts()

weighting_scheme
mean         45
attention    45
Name: count, dtype: int64

In [43]:
df_final[df_final['scale_guide_loss'] == 'False']

,val_epoch,val_epoch__std,weighting_scheme,alpha,fine_tune_epoch,reconstruct_loss,reconstruction_layer,weight_scale,use_down_scale,use_upscale,...,scale_guide_loss,user_profile_embeddings_files,val_NDCG@10,val_NDCG@10__std,val_Recall@10,val_Recall@10__std,test_NDCG@10,test_NDCG@10__std,test_Recall@10,test_Recall@10__std
params,,,,,,,,,,,,,,,,,,,,,
"(mean, 0.5, 6, MSE, 2, 0.1, True, False, False, mean, False, gemma_short_large_umap_single)",9.000000,0.000000,mean,0.5,6.0,MSE,2.0,0.1,True,False,...,False,gemma_short_large_umap_single,0.023560,0.000114,0.045833,0.000167,0.010613,0.000415,0.022747,0.000685
"(mean, 0.5, 6, RMSE, 1, 0.1, True, False, False, mean, False, gemma_short_large_umap_single)",8.666667,0.577350,mean,0.5,6.0,RMSE,1.0,0.1,True,False,...,False,gemma_short_large_umap_single,0.023210,0.000332,0.045190,0.001808,0.010583,0.000435,0.022547,0.001066
"(attention, 0.7, 6, MSE, 1, 0.1, True, False, False, mean, False, gemma_long_large_single)",8.333333,1.527525,attention,0.7,6.0,MSE,1.0,0.1,True,False,...,False,gemma_long_large_single,0.023493,0.000313,0.046527,0.001419,0.010580,0.000401,0.022507,0.000856
"(attention, 0.5, 6, RMSE, 1, 0.1, True, False, False, mean, False, gemma_long_small_single)",9.666667,0.577350,attention,0.5,6.0,RMSE,1.0,0.1,True,False,...,False,gemma_long_small_single,0.023437,0.000418,0.046380,0.000397,0.010577,0.000172,0.022550,0.000308
"(mean, 0.5, 6, MSE, 2, 0.1, True, False, False, mean, False, gemma_long_large_umap_single)",8.666667,0.577350,mean,0.5,6.0,MSE,2.0,0.1,True,False,...,False,gemma_long_large_umap_single,0.023687,0.000535,0.046283,0.001561,0.010537,0.000355,0.022470,0.000334
"(mean, 0.7, 12, RMSE, 1, 0.1, True, False, False, mean, False, gemma_short_large_umap_single)",9.000000,0.000000,mean,0.7,12.0,RMSE,1.0,0.1,True,False,...,False,gemma_short_large_umap_single,0.023563,0.000221,0.046527,0.000391,0.010507,0.000483,0.022727,0.000745
"(attention, 0.5, 6, MSE, 2, 0.1, True, False, False, mean, False, gemma_long_small_single)",10.000000,1.000000,attention,0.5,6.0,MSE,2.0,0.1,True,False,...,False,gemma_long_small_single,0.023577,0.000717,0.045637,0.001853,0.010477,0.000424,0.022390,0.001345
"(attention, 0.5, 12, RMSE, 1, 0.1, True, False, False, mean, False, gemma_long_large_umap_single)",10.333333,0.577350,attention,0.5,12.0,RMSE,1.0,0.1,True,False,...,False,gemma_long_large_umap_single,0.023540,0.000215,0.046977,0.001179,0.010443,0.000346,0.022390,0.000711
"(attention, 0.7, 12, MSE, 1, 0.1, True, False, False, mean, False, gemma_long_large_single)",8.333333,1.527525,attention,0.7,12.0,MSE,1.0,0.1,True,False,...,False,gemma_long_large_single,0.023597,0.000616,0.047523,0.001644,0.010443,0.000170,0.022430,0.000606


In [40]:
df['params'].value_counts()

params
(mean, 0.7, 12, MSE, 1, 0.1, True, False, False, mean, True, gemma_short_large_umap_single)         3
(mean, 0.7, 12, MSE, 2, 0.1, True, False, False, mean, False, gemma_long_large_umap_single)         3
(mean, 0.6, 6, RMSE, 2, 0.1, True, False, False, mean, True, gemma_long_large_umap_single)          3
(attention, 0.5, 6, RMSE, 1, 0.1, True, False, False, mean, False, gemma_long_large_umap_single)    3
(mean, 0.7, 12, MSE, 2, 0.1, True, False, False, mean, False, gemma_long_large_single)              3
                                                                                                   ..
(mean, 0.6, 12, MSE, 1, 0.1, True, False, False, mean, False, gemma_long_large_umap_single)         1
(mean, 0.7, 6, RMSE, 1, 0.1, False, False, False, mean, True, gemma_long_large_umap_single)         1
(attention, 0.5, 6, MSE, 2, 0.1, True, False, False, mean, False, gemma_long_large_single)          1
(attention, 0.6, 6, MSE, 2, 0.1, False, False, False, mean, True, gemma_lon

In [18]:
df['']

,config_file,weighting_scheme,alpha,fine_tune_epoch,reconstruct_loss,reconstruction_layer,weight_scale,use_down_scale,use_upscale,multi_profile,...,user_profile_embeddings_files,seed,val_NDCG@10,val_Recall@10,val_epoch,test_NDCG@10,test_Recall@10,test_epoch,params,all_seeds
663,mean-0.8-6-RMSE-1-0.1-False-True-False-mean-Fa...,mean,0.80,6,RMSE,1,0.1,False,True,False,...,gemma_short_large_umap_single,42,0.09987,0.18422,14,0.06111,0.11661,14,"(mean, 0.8, 6, RMSE, 1, 0.1, False, True, Fals...",[42]
837,exponential-0.8-8-MSE-2-0.1-False-True-False-m...,exponential,0.80,8,MSE,2,0.1,False,True,False,...,gemma_long_large_umap_single,42,0.09898,0.18188,14,0.06083,0.11617,14,"(exponential, 0.8, 8, MSE, 2, 0.1, False, True...",[42]
779,exponential-0.65-8-RMSE-2-0.1-False-True-False...,exponential,0.65,8,RMSE,2,0.1,False,True,False,...,gemma_short_large_umap_single,42,0.09998,0.18356,14,0.06072,0.11579,14,"(exponential, 0.65, 8, RMSE, 2, 0.1, False, Tr...",[42]
649,mean-0.8-6-MSE-2-0.1-False-True-False-mean-Tru...,mean,0.80,6,MSE,2,0.1,False,True,False,...,gemma_long_large_umap_single,42,0.09983,0.18389,14,0.06072,0.11615,14,"(mean, 0.8, 6, MSE, 2, 0.1, False, True, False...",[42]
467,exponential-0.5-8-RMSE-2-0.1-True-False-False-...,exponential,0.50,8,RMSE,2,0.1,True,False,False,...,gemma_short_large_umap_single,42,0.09985,0.18454,14,0.06072,0.11703,14,"(exponential, 0.5, 8, RMSE, 2, 0.1, True, Fals...",[42]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,exponential-0.8-8-RMSE-2-0.1-True-False-False-...,exponential,0.80,8,RMSE,2,0.1,True,False,False,...,gemma_long_large_single,42,0.09951,0.18465,19,0.05589,0.10964,19,"(exponential, 0.8, 8, RMSE, 2, 0.1, True, Fals...",[42]
792,exponential-0.8-4-MSE-1-0.1-False-True-False-m...,exponential,0.80,4,MSE,1,0.1,False,True,False,...,gemma_long_large_single,42,0.09851,0.18318,11,0.05586,0.10847,11,"(exponential, 0.8, 4, MSE, 1, 0.1, False, True...",[42]
25,attention-0.65-8-MSE-1-0.1-False-True-False-me...,attention,0.65,8,MSE,1,0.1,False,True,False,...,gemma_long_large_umap_single,42,0.09773,0.18112,18,0.05583,0.10835,18,"(attention, 0.65, 8, MSE, 1, 0.1, False, True,...",[42]
274,attention-0.8-6-MSE-2-0.1-True-False-False-mea...,attention,0.80,6,MSE,2,0.1,True,False,False,...,gemma_short_large_single,42,0.09796,0.18210,7,0.05579,0.10824,7,"(attention, 0.8, 6, MSE, 2, 0.1, True, False, ...",[42]


In [17]:
df[PARAMS[1:]].to_json('best_kion.json', index=False, orient="records")

In [42]:
df_final[PARAMS[1:]].to_json('best_beauty_3seeds.json', index=False, orient="records")

In [17]:
results.keys()

dict_keys(['-2_0/configs/sasrec/beauty/BEAUTY_INITIAL/single_seed/attention-0.7-12-MSE-2-0.1-False-False-False-mean-False-gemma_long_large_umap_single-42.yaml', '-2_0/configs/sasrec/beauty/BEAUTY_INITIAL/single_seed/attention-0.7-12-MSE-2-0.1-False-False-False-mean-False-gemma_short_large_umap_single-42.yaml', '-2_0/configs/sasrec/beauty/BEAUTY_INITIAL/single_seed/attention-0.7-12-MSE-2-0.1-False-False-False-mean-True-gemma_long_large_umap_single-42.yaml', '-2_0/configs/sasrec/beauty/BEAUTY_INITIAL/single_seed/attention-0.7-12-MSE-2-0.1-False-False-False-mean-True-gemma_short_large_umap_single-42.yaml', '-2_0/configs/sasrec/beauty/BEAUTY_INITIAL/single_seed/attention-0.7-12-MSE-2-0.1-True-False-False-mean-False-gemma_long_large_single-42.yaml', '-2_0/configs/sasrec/beauty/BEAUTY_INITIAL/single_seed/attention-0.7-12-MSE-2-0.1-True-False-False-mean-False-gemma_long_large_umap_single-42.yaml', '-2_0/configs/sasrec/beauty/BEAUTY_INITIAL/single_seed/attention-0.7-12-MSE-2-0.1-True-False-Fal

In [23]:
results['-2_0/configs/sasrec/beauty/BEAUTY_INITIAL/single_seed/attention-0.7-12-MSE-2-0.1-False-False-False-mean-False-gemma_long_large_umap_single-42.yaml']

{'validation_metrics': [{'epoch': 1,
   'loss': 10.828,
   'Recall@5': 0.004162330905306972,
   'NDCG@5': 0.0023390187078764875,
   'Recall@10': 0.006243496357960458,
   'NDCG@10': 0.00302255539974203,
   'Recall@20': 0.010851791288836034,
   'NDCG@20': 0.004198139363666408,
   'loss_recsys': np.float64(10.347532872800473),
   'loss_guide': np.float64(0.0)},
  {'epoch': 2,
   'loss': 5.2138,
   'Recall@5': 0.004013676230117437,
   'NDCG@5': 0.0024368647296024288,
   'Recall@10': 0.008324661810613945,
   'NDCG@10': 0.003826736836797046,
   'Recall@20': 0.014122194143005798,
   'NDCG@20': 0.0052561474396673165,
   'loss_recsys': np.float64(10.325713298938892),
   'loss_guide': np.float64(0.0)},
  {'epoch': 3,
   'loss': 4.8,
   'Recall@5': 0.006094841682770923,
   'NDCG@5': 0.003388410317555781,
   'Recall@10': 0.009216589861751152,
   'NDCG@10': 0.004382003442624782,
   'Recall@20': 0.015014122194143005,
   'NDCG@20': 0.005846106928100518,
   'loss_recsys': np.float64(10.315859370761448

In [22]:
df['config_file']

426    mean-0.7-12-MSE-1-0.1-True-False-False-mean-Tr...
11     attention-0.7-12-MSE-2-0.1-True-False-False-me...
88     mean-0.7-6-RMSE-1-0.1-True-False-False-mean-Tr...
283    mean-0.5-6-MSE-1-0.1-True-False-False-mean-Tru...
373    attention-0.5-12-MSE-1-0.1-True-False-False-me...
                             ...                        
73     mean-0.7-6-MSE-2-0.1-True-False-False-mean-Tru...
257    mean-0.7-12-MSE-2-0.1-True-False-False-mean-Tr...
234    mean-0.6-6-MSE-2-0.1-True-False-False-mean-Tru...
259    mean-0.7-12-MSE-2-0.1-True-False-False-mean-Tr...
2      attention-0.7-12-MSE-2-0.1-False-False-False-m...
Name: config_file, Length: 558, dtype: object

In [1]:
PARAMS

NameError: name 'PARAMS' is not defined

In [47]:
df_final.to_csv('beauty_llm_runs.csv')